In [36]:
import time
start = time.time()

In [37]:
import sys 
import os
import torchvision
import torchvision.transforms as transforms
import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, datasets

def read_folder(path):
    files = os.listdir(path)
    for name in files:
        if name.find(' ') != -1:
            os.rename(path+'/' + name, path+ '/' +name.replace(' ', '_'))



In [38]:
path_train  = './fruits_360/fruits-360/Training'
path_test = './fruits_360/fruits-360/Test'

read_folder(path_train)
read_folder(path_test)

In [39]:

train_dataset = datasets.ImageFolder(path_train, transform = transforms.ToTensor())
train_loader = DataLoader(train_dataset, batch_size = 4, shuffle = True)

test_dataset = datasets.ImageFolder(path_test, transform = transforms.ToTensor())
test_loader = DataLoader(test_dataset, batch_size = 4, shuffle = True)


In [40]:
class Net(nn.Module):

    def __init__(self):

        super(Net, self).__init__()
        # 1 input image channel, 6 output channels, 5x5 square convolution kernel
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        # an affine operation: y = Wx + b
        self.fc1 = nn.Linear(16 * 22 * 22, 120) # (size of input, size of output)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 83)



    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x= self.dropout(x)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):

        size = x.size()[1:] # all dimensions except the batch dimension

        num_features = 1
        for s in size:
    
            num_features *= s

        return num_features



In [41]:

#check CPU on machine 

device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")

if device == torch.device('cuda:0') :
    print "Device : cuda => ok"
else :
    print "Device : cpu"
    print "CUDA required"

net = Net().to(device) # for cuda
#net = Net()
print(net)

criterion = nn.CrossEntropyLoss()
optimizer=optim.SGD(net.parameters(),lr=0.0001,momentum=0.9)

Device : cuda => ok
Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=7744, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=83, bias=True)
)


In [42]:
epochs = 4
for epoch in range(epochs):
    running_loss=0.0
    for i, data in enumerate(train_loader,0):
        inputs,labels = data
        inputs,labels = inputs.to(device), labels.to(device) # for cuda
        optimizer.zero_grad()
        outputs=net(inputs)
        loss=criterion(outputs,labels)
        loss.backward()
        optimizer.step()

        running_loss+=loss.item()
        if i % 2000 == 1999: 
            print ('[%d,%5d] loss: %.3f' % (epoch+1,i+1,running_loss/2000))
            running_loss=0.0
print ('Finished Training')

[1, 2000] loss: 4.415
[1, 4000] loss: 4.300
[1, 6000] loss: 3.122
[1, 8000] loss: 2.162
[1,10000] loss: 1.622
[2, 2000] loss: 1.018
[2, 4000] loss: 0.719
[2, 6000] loss: 0.553
[2, 8000] loss: 0.444
[2,10000] loss: 0.363
[3, 2000] loss: 0.291
[3, 4000] loss: 0.235
[3, 6000] loss: 0.194
[3, 8000] loss: 0.192
[3,10000] loss: 0.164
[4, 2000] loss: 0.114
[4, 4000] loss: 0.134
[4, 6000] loss: 0.089
[4, 8000] loss: 0.091
[4,10000] loss: 0.079
Finished Training


In [43]:

def imshow(img):
    img=img/2+0.5
    npimg=img.numpy()
    plt.imshow(np.transpose(npimg,(1,2,0)))

testiter = iter(test_loader)
images, labels =  testiter.next()
print labels

tensor([56, 82, 12, 25])


In [44]:
images = images.to(device, torch.float)
outputs = net(images)
_,predicted = torch.max(outputs,1)
print predicted

tensor([56, 82, 12, 25], device='cuda:0')


In [45]:
correct = 0
total = 0
with torch.no_grad():
    for data in test_loader:
        images, labels = data # cuda ?
        images = images.to(device, torch.float)
        labels = labels.to(device, torch.long)
        outputs=net(images)
        _,predicted= torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print "done"

done


In [46]:
print correct
print total
print (100*correct)/int(total)
    
print 'Test dataset : Accuracy of the network on the %(tot)s test images %(percent)s.%(rest)s %%' % {'tot':total, 'percent':((100*correct)/int(total)), 'rest':((100*correct)%int(total)) }


12093
14369
84
Test dataset : Accuracy of the network on the 14369 test images 84.2304 %


In [47]:
correct = 0
total = 0
with torch.no_grad():
    for data in train_loader:
        images, labels = data # cuda ?
        images = images.to(device, torch.float)
        labels = labels.to(device, torch.long)
        outputs=net(images)
        _,predicted= torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        
print "done"

done


In [48]:
print correct
print total
print (100*correct)/int(total)
    
print 'Training dataset : Accuracy of the network on the %(tot)s training images %(percent)s.%(rest)s %%' % {'tot':total, 'percent':((100*correct)/int(total)), 'rest':((100*correct)%int(total)) }


41733
42798
97
Training dataset : Accuracy of the network on the 42798 training images 97.21894 %


In [49]:
#Time counter
print "Time elapsed :", time.time() - start, "s"

Time elapsed : 215.169229984 s
